<a href="https://colab.research.google.com/github/satuelisa/DBforWeb/blob/main/DBW_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/satuelisa/DBforWeb/main/courses.csv' # another example data, a bit larger

# if we do not want the course numbers to lose their leading zeroes, we can force everything into strings
conv = { 'Number': lambda x: str(x) } # the amounts can be float, that is not a problem
csv = pd.read_csv(url, converters = conv)
print(csv.dtypes)
csv # they still act like numbers when viewed, though

Code       object
Number     object
Title      object
Amount    float64
Kind       object
dtype: object


,Code,Number,Title,Amount,Kind
0,YCBS,255,Statistical Machine Learning,8.0,CEUs
1,YCBS,256,Data Science for Business Decisions,8.0,CEUs
2,YCBS,257,Data at Scale,9.0,CEUs
3,YCBS,258,Practical Machine Learning,9.0,CEUs
4,YCBS,260,Statistics for Business Decision Making,8.0,CEUs
...,...,...,...,...,...
66,YCBS,242,Applied Business Analysis,4.5,CEUs
67,YCBS,243,Introduction to Business Analytics and Data Mo...,4.0,CEUs
68,YCBS,216,Interpersonal Skills for Professionals,4.5,CEUs
69,YCBS,221,Leading Sustainable Change,6.0,CEUs


In [ ]:
import sqlite3
fields = ','.join(csv.columns)
connector = sqlite3.connect('example.db')
cursor = connector.cursor()
cursor.execute(f'CREATE TABLE example ({fields});') 
rows = csv.to_dict(orient = 'split')['data']
wildcard = ('?,' * len(csv.columns))[:-1]
cursor.executemany(f'INSERT INTO example ({fields}) VALUES ({wildcard});', rows) 
connector.commit()
connector.close()

That is what we learned to do in Session 1. Today, we will work on making more *queries* on a **single** table. Next session, we will learn to combine information from two or more tables.

All of this makes more sense if you take a look at at the [data](https://github.com/satuelisa/DBforWeb/blob/main/courses.csv) before continuing.

In [ ]:
connector = sqlite3.connect('example.db')
cursor = connector.cursor()
cursor = connector.execute('SELECT Code || printf("%03d", Number) FROM example WHERE Code = "YCIT";') # concatenate if matching, enforce the leading zeroes back
contents = cursor.fetchall()
print(contents)
connector.close()

[('YCIT017',), ('YCIT018',), ('YCIT019',), ('YCIT020',), ('YCIT021',), ('YCIT022',), ('YCIT001',), ('YCIT002',), ('YCIT003',), ('YCIT004',), ('YCIT005',), ('YCIT006',), ('YCIT007',), ('YCIT008',), ('YCIT009',), ('YCIT010',), ('YCIT011',), ('YCIT012',), ('YCIT013',), ('YCIT014',), ('YCIT015',), ('YCIT016',)]


Next up, we have a [`CASE` statement](https://www.sqlitetutorial.net/sqlite-case/).

In [ ]:
connector = sqlite3.connect('example.db')
cursor = connector.cursor()
cursor = connector.execute('''SELECT Title, Amount,
    CASE 
       WHEN Amount <= 3 THEN 'regular'
       WHEN Amount > 3 THEN 'heavy'
    END 
  AS scale 
FROM example;''') # case by case mapping
contents = cursor.fetchall()
for c in contents:
  print(c)
connector.close()

('Statistical Machine Learning', 8.0, 'heavy')
('Data Science for Business Decisions', 8.0, 'heavy')
('Data at Scale', 9.0, 'heavy')
('Practical Machine Learning', 9.0, 'heavy')
('Statistics for Business Decision Making', 8.0, 'heavy')
('Data Analytics Fundamentals', 8.0, 'heavy')
('Leading Data Science Projects & Teams', 8.0, 'heavy')
('Data Science Capstone Project', 9.0, 'heavy')
('Cloud Computing Fundamentals', 7.5, 'heavy')
('Cloud Networking and Security', 7.5, 'heavy')
('Cloud Architecture', 9.0, 'heavy')
('Advanced Cloud Architecture', 9.0, 'heavy')
('DevOps Practices and Tools', 9.0, 'heavy')
('Site Reliability Engineering (SRE)', 9.0, 'heavy')
('Predictive & Classification Modelling', 6.0, 'heavy')
('Neural Networks & Deep Learning', 6.0, 'heavy')
('Intelligent Agents & Reinforcement Learning', 6.0, 'heavy')
('Deep Learning for Computer Vision', 6.0, 'heavy')
('Natural Language Processing Fundamentals', 6.0, 'heavy')
('Time Series Analysis Fundamentals', 6.0, 'heavy')
('Inter

How about picking just a few matches?

In [ ]:
connector = sqlite3.connect('example.db')
cursor = connector.cursor()
cursor = connector.execute('SELECT * FROM example LIMIT 7;') #  first seven
contents = cursor.fetchall()
for c in contents:
  print(c)
connector.close()

('YCBS', '255', 'Statistical Machine Learning', 8.0, 'CEUs')
('YCBS', '256', 'Data Science for Business Decisions', 8.0, 'CEUs')
('YCBS', '257', 'Data at Scale', 9.0, 'CEUs')
('YCBS', '258', 'Practical Machine Learning', 9.0, 'CEUs')
('YCBS', '260', 'Statistics for Business Decision Making', 8.0, 'CEUs')
('YCBS', '261', 'Data Analytics Fundamentals', 8.0, 'CEUs')
('YCBS', '262', 'Leading Data Science Projects & Teams', 8.0, 'CEUs')


In [ ]:
connector = sqlite3.connect('example.db')
cursor = connector.cursor()
cursor = connector.execute('SELECT * FROM example ORDER BY random() LIMIT 7;') # random sample
contents = cursor.fetchall()
for c in contents:
  print(c)
connector.close()

('YCNG', '233', 'Time Series Analysis Fundamentals', 6.0, 'CEUs')
('CCCS', '425', 'Web Services', 3.0, 'credits')
('YCBS', '216', 'Interpersonal Skills for Professionals', 4.5, 'CEUs')
('YCIT', '4', 'JavaScript Libraries', 4.0, 'CEUs')
('CCCS', '451', 'Communication and Network Security', 3.0, 'credits')
('YCNG', '230', 'Intelligent Agents & Reinforcement Learning', 6.0, 'CEUs')
('YCIT', '7', 'JavaScript Frameworks', 4.0, 'CEUs')


In [ ]:
connector = sqlite3.connect('example.db')
cursor = connector.cursor()
cursor = connector.execute('SELECT * FROM example ORDER BY Amount LIMIT 7;') # increasing order
contents = cursor.fetchall()
for c in contents:
  print(c)
connector.close()

('CCCS', '300', 'Programming Techniques 1', 3.0, 'credits')
('CCCS', '301', 'Programming Techniques 2', 3.0, 'credits')
('CCCS', '310', 'Web Development', 3.0, 'credits')
('CCCS', '315', 'Data Structures and Algorithms', 3.0, 'credits')
('CCCS', '321', 'Operating Systems Administration', 3.0, 'credits')
('CCCS', '325', 'Mobile Application Development', 3.0, 'credits')
('CCCS', '330', 'Database Design and Business Applications Development', 3.0, 'credits')


In [ ]:
connector = sqlite3.connect('example.db')
cursor = connector.cursor()
cursor = connector.execute('SELECT * FROM example ORDER BY Amount DESC LIMIT 7;') # decreasing order
contents = cursor.fetchall()
for c in contents:
  print(c)
connector.close()

('YCBS', '210', 'Comprehensive Business Analysis', 16.0, 'CEUs')
('CBUS', '4', 'Special Topics in Information Technology', 14.0, 'credits')
('YCBS', '257', 'Data at Scale', 9.0, 'CEUs')
('YCBS', '258', 'Practical Machine Learning', 9.0, 'CEUs')
('YCBS', '299', 'Data Science Capstone Project', 9.0, 'CEUs')
('YCIT', '19', 'Cloud Architecture', 9.0, 'CEUs')
('YCIT', '20', 'Advanced Cloud Architecture', 9.0, 'CEUs')
